In [1]:
!pip install deeplake

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import deeplake
from PIL import Image
import numpy as np
import os, time
import torch
from torchvision import transforms
import torch.nn as nn
from torchvision.models import resnet50, ResNet50_Weights
import matplotlib.pyplot as plt

In [3]:
#only run this if youre on colab
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
ds_train = deeplake.load("hub://activeloop/stanford-cars-train")
ds_test = deeplake.load("hub://activeloop/stanford-cars-test")

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/stanford-cars-train



\

hub://activeloop/stanford-cars-train loaded successfully.



/

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/stanford-cars-test



|

hub://activeloop/stanford-cars-test loaded successfully.



In [5]:
ds_train.summary()

Dataset(path='hub://activeloop/stanford-cars-train', read_only=True, tensors=['images', 'car_models', 'boxes'])

   tensor       htype                 shape               dtype  compression
  -------      -------               -------             -------  ------- 
   images       image     (8144, 57:3744, 78:5616, 1:3)   uint8    jpeg   
 car_models  class_label            (8144, 1)            uint32    None   
   boxes        bbox              (8144, 1, 4)           float32   None   


In [12]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7,7), stride=(2,2), padding=(3,3), bias=False)
model.fc = torch.nn.Linear(model.fc.in_features, len(ds_train.car_models))
model.fc.requires_grad_(True)
model.to(device)

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
for x in model.parameters():
  if x.requires_grad:
    print(x)
if model.fc.requires_grad_:
  print("Unfrozen FC Layer")

In [7]:
batch_size = 100
num_epochs = 5
learning_rate = 0.003

# Loss Function
loss_fn = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [8]:
tform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((240,240)),
    transforms.ToTensor(), # Must convert to pytorch tensor for subsequent operations to run
    transforms.Normalize((0.5), (0.5)),
])

train_loader = ds_train.pytorch(num_workers = 0, shuffle = False, transform = {'images': tform, 'car_models': None}, batch_size = batch_size, tensors = ['images', 'car_models'], decode_method = {'images': 'pil'})
test_loader = ds_test.pytorch(num_workers = 0, transform = {'images': tform, 'car_models': None}, batch_size = batch_size, tensors = ['images', 'car_models'], decode_method = {'images': 'pil'})

In [9]:
def train(model, loss_fn, optimizer, train_loader, batch_size, num_epochs, device=None):
    model.train()
    for epoch in range(1, num_epochs+1):
      running_loss = 0.0
      start_time = time.time()
      total = 0
      correct = 0
      for i, data in enumerate(train_loader):
            inputs, labels = data['images'], torch.squeeze(data['car_models'])
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad() #zero gradient optimizer
            outputs = model(inputs) #get outputs from current model's weights
            loss = loss_fn(outputs, labels) #get loss 
            loss.backward() #back prop
            optimizer.step() #optimize
            _, predicted = torch.max(outputs.data, 1) #get predictions from model
            total += labels.size(0) #statistics stuff
            correct += (predicted == labels).sum().item() #statistics stuff
            accuracy = 100 * correct / total #find epoch acc

            running_loss += loss.item() #find loss
            if i % batch_size == 0: #per batch, print statistics
              batch_time = time.time()
              speed = (i+1)/(batch_time-start_time)
              print('[%5d] loss: %.3f, speed: %.2f, accuracy: %.2f %%' % (i, running_loss, speed, accuracy))
              running_loss, total, correct = 0, 0, 0

In [10]:
def test(model, test_loader, device=None):
  model.eval()
  start_time = time.time()
  correct = 0
  total = 0
  with torch.no_grad():
    for i, data in enumerate(test_loader):
      inputs, labels = data['images'], torch.squeeze(data['car_models'])
      inputs, labels = inputs.to(device), labels.to(device)
      optimizer.zero_grad()
      outputs = model(inputs)
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total

    print('Testing Accuracy: %.2f %%' %(accuracy))

In [ ]:
train(model = model, loss_fn = loss_fn, optimizer = optimizer, batch_size = batch_size, num_epochs = num_epochs, device = device, train_loader = train_loader)

[    0] loss: 8.949, speed: 0.13, accuracy: 0.00 %
[    0] loss: 8.949, speed: 0.14, accuracy: 0.00 %
[    0] loss: 8.949, speed: 0.11, accuracy: 0.00 %
[    0] loss: 8.949, speed: 0.11, accuracy: 0.00 %
[    0] loss: 8.949, speed: 0.25, accuracy: 0.00 %


In [ ]:
test(model = model, test_loader = test_loader, device = device)
